In [1]:
import csv, datetime, re

SUBS = "Monthly Bills: Subscriptions"
GRC = "Everyday Expenses: Groceries"
FUN = "Fun Stuff: UNK"
TRNSP = "Everyday Expenses: Transportation"
HSHLD = "Everyday Expenses: Household Goods"
REST = "Everyday Expenses: Restaurants"
MED = "Everyday Expenses: Medical"
GAS = "Everyday Expenses: Gas"
MOVIES = "Fun Stuff: Movies"
BANKING = "Monthly Bills: Banking"
ELECTRONICS = "Fun Stuff: Gadgets"
MUSIC = "Fun Stuff: Music"
PHONE ="Monthly Bills: Phone"
SPORTS = "Fun Stuff: Sports"
INSURANCE = "Rainy Day Funds: Insurance"
PARKING = "Monthly Bills: Parking"
BOOKS = "Fun Stuff: Books"
CLOTHING = "Everyday Expenses: Clothing"

In [19]:
regex1 = "Voucher: \d* / Tranzactie comerciant - Tranz: Nr card [^ ]*, Ref \d*, Suma platita \d*.\d* RON. Comision: 0 RON. Locatie: [^ ]* (?:(?:RO|OM|DE|A|DU) )?(.*) Data_Ora: .*"
regex_foreign = "Voucher: \d* / Tranzactie comerciant - Tranz: Nr card [^ ]*, Ref \d*, Suma platita \d*.\d* (?:EUR|USD), Suma in valuta contului \d*.\d* RON, Suma decontata \d*.\d* (?:EUR|USD) Rata de schimb .* RON. Comision: .* RON. Locatie: (.*). Data_Ora:"
regex_income = "Voucher: \d* \/ Referinta [^,]*, data valutei \d\d-\d\d-\d\d\d\d, Decontare -Platitor: (.*); .*Beneficiar: .*; RO53RNCB0035103924780001-Detalii: .*"
f = open("./bcr4.csv")

expense_reader = csv.reader(f)
expenses = list(expense_reader)

#NOTICe
# MCH ORADEA might be KFC
processors = [("NIS ORADEA", "Gazprom", GAS),
              ("LIDL", "Lidl", GRC),
              ("PH LOTUS", "Pizza Hut", REST),
              ("CINEMATOGRAF LOTUS", "Cinema Palace", MOVIES),
              ("Comision Exchange", "BCR", BANKING),
              ("Amazon", "Amazon", FUN),
              ("MEDIAGALAXY", "MediaGalaxy", ELECTRONICS),
              ("ALTEX", "Altex", ELECTRONICS),
              ("CARREFOUR", "Carrefour", GRC),
              ("Retragere Numerar", "Transfer: Cash", ""),
              ("NETFLIX.COM", "Netflix", SUBS),
              ("MOL ORADEA", "Mol", GAS),
              ("BANDCAMP", "Bandcamp", MUSIC),
              ("TECHONA", "Meatic", REST),
              ("JUMBO", "Jumbo", HSHLD),
              ("WWW.ORANGE.RO", "Orange", PHONE),
              ("AUCHAN", "Auchan", GRC),
              ("ROUMASPORT", "Decathlon", SPORTS),
              ("eMAG.ro", "eMag", ELECTRONICS),
              ("US FOOD NETWORK", "KFC", REST),
              ("WWW.DECLIC.RO", "DeClic", "Giving: Charitable"),
              ("INST NAT SI DEZVOLTARE BUCURESTI.", "ROTLD", SUBS),
              ("CFR UNATTENDED UNATTENDED", "CFR", TRNSP),
              ("Comision Card", "BCR", BANKING),
              ("LIBRARIILE HUMANITAS", "Humanitas", BOOKS),
              ("Primaria Municipiul Or Oradea", "Primaria Oradea", PARKING),
              ("KAUFLAND", "Kaufland", GRC), 
              ("OMV", "OMV", GAS),
              ("UNIQAONLINE", "UNIQA", INSURANCE),
              ("DEDEMAN", "Dedeman", HSHLD),
              ("Comision Primire", "BCR", BANKING),
              ("Funding", "BCR", BANKING),
              ("ARTIMA", "Carrefour", GRC),
              ("H&M", "H&M", CLOTHING),
              ("SENSIBLU", "Sensiblu", MED),
              ("AVSTORE BUCURESTI", "AVStore", ELECTRONICS),
              ("ANDROMI", "Andromi", GRC),
              ("ORANGE ROMANIA", "Orange", PHONE),
              ("PACT EXIM", "Saladbox",REST),
              ("SCRIPTUM", "Scriptum", BOOKS),
              ("ACTIV FOOD EXPRES", "Seramis Coffee", GRC),
              ("SUBWAY ERA", "Subway", REST),
              ("MEDLENS", "Medlens", MED),
              ("GOOGLE *Google Music g.co/helppay", "Google", SUBS),
              ("KFC ORADEA DT ORADEA", "KFC", REST),
              ("JYSK", "Jysk", HSHLD),
              ("BEEMINDER PORTLAND", "Beeminder", SUBS)
             ]
new_expenses = []
for expense in expenses:
    date, _, payee, outflow = expense
    
    for expr in [regex1, regex_foreign, regex_income]:
        reg = re.match(expr, payee)
        if reg:
            break
    if reg is None:
        if "Retragere numerar" in payee:
            payee = "Retragere Numerar"
        elif "Incasare Comision de administrare pachet de produse si servicii pt contul" in payee:
            payee = "Comision Card"
        elif "Comision conversie pentru tranzactia cu voucherul" in payee:
            payee = "Comision Card"
        elif "Incasari interbancare in lei" in payee:
            payee = "Comision Card"
        else:
            print(payee)
            continue
    else:
        payee = reg.group(1)
    category = ''
    memo = []
    outflow = outflow.replace(".", "").replace(",",".").replace(" RON", "")
    if "Currency rate" in payee:
        pass
    for substr, name, cat in processors:
        if substr in payee:
            payee = name
            category = cat
            break
    else:
        print(payee)
    if float(outflow) > 150:
        memo.append("expensive")

    new_expenses.append([date, payee, category, ",".join(memo), -float(outflow), 0])
    

for expense in new_expenses:
    print(expense)
with open('cleaned.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Date", "Payee", "Category", "Memo", "Outflow", "Inflow"])
    writer.writerows(new_expenses)

MCH ORADEA.
KITCHEN SHOP Bucuresti.
['11.05.2018', 'Kaufland', 'Everyday Expenses: Groceries', '', 69.01, 0]
['10.05.2018', 'Kaufland', 'Everyday Expenses: Groceries', '', 14.22, 0]
['09.05.2018', 'Jumbo', 'Everyday Expenses: Household Goods', '', 39.95, 0]
['08.05.2018', 'MCH ORADEA.', '', '', 42.4, 0]
['07.05.2018', 'Lidl', 'Everyday Expenses: Groceries', '', 13.88, 0]
['07.05.2018', 'Decathlon', 'Fun Stuff: Sports', '', 115.3, 0]
['06.05.2018', 'Auchan', 'Everyday Expenses: Groceries', '', 46.0, 0]
['04.05.2018', 'Kaufland', 'Everyday Expenses: Groceries', '', 9.78, 0]
['30.04.2018', 'Carrefour', 'Everyday Expenses: Groceries', '', 29.88, 0]
['23.04.2018', 'Kaufland', 'Everyday Expenses: Groceries', '', 40.76, 0]
['20.04.2018', 'Kaufland', 'Everyday Expenses: Groceries', '', 49.45, 0]
['16.04.2018', 'Carrefour', 'Everyday Expenses: Groceries', '', 49.93, 0]
['09.04.2018', 'KITCHEN SHOP Bucuresti.', '', '', 197.0, 0]


In [ ]:

f = open("./viseca7.csv")

expense_reader = csv.reader(f)
expenses = list(expense_reader)

processors = [("NETFLIX", "Netflix", SUBS), 
              ("Google Commerc", "Google", SUBS),
              ("GOOGLE *Commerc", "Google", SUBS),
              ("GOOGLE *Google Music", "Google", SUBS),
              ("GOOGLE *CDG,", "Google", SUBS),
              ("Coop-3305, Langnau am Al", "Coop Langnau", GRC),
              ("DIGITALOCEAN.COM", "DigitalOcean", SUBS),
              ("Coop", "Coop", GRC),
              ("Migros", "Migros", GRC),
              ("BOUNTYSOURCE", "Bountysource", SUBS),
              ("Amazon", "Amazon", FUN),
              ("AMAZON", "Amazon", FUN),
              ("GOOGLE *Voice", "Google", "Monthly Bills: Internet"), 
              ("GOOGLE *Google Storage", "Google", "Monthly Bills: Internet"), 
              ("UBER", "Uber", TRNSP),
              ("Uber", "Uber", TRNSP),
              ("WP-IFES TRUST", "Debora Roehl", "Giving: Charitable"),
              ("LUFTHANSA", "Lufthansa", TRNSP),
              ("IKEA", "IKEA", HSHLD),
              ("Gidor Coiffure", "Gidor Sihlcity", "Everyday Expenses: Clothing"),
              ("UDACITY, INC.", "Udacity", "Monthly Bills: Subscriptions"),
              ("BEEMINDER", "Beeminder", "Monthly Bills: Subscriptions"),
              ("ITUNES.COM/BILL", "iTunes", FUN),
              ("AIRBNB", "Airbnb", "Fun Stuff: Trips"),
              ("Manor AG - 229, Zuerich", "Manor", REST),
              ("ZUERICH FLUGHAFEN SBB, ZUERICH FLUG", "SBB", TRNSP), 
              ("WWW.ORANGE.RO, BUCURESTI", "Orange", SUBS),
              ("Hiltl Sihlpost, Zurich", "Hiltl", REST),
              ("Swiss smile Schweiz AG", "Swiss Smile", MED),
              ("SCHWEIZ. BUNDESBAHN, BERN", "SBB", TRNSP),
              ("VBZ SCHMIEDE WIEDIKON, ZUERICH", "SBB", TRNSP),
              ("SBB CFF FSS Ticket Shop", "SBB", TRNSP),
              ("ZVV-TICKET-APP, ZUERICH", "SBB", TRNSP), 
              ("PAYPAL *CASAJURNALI", "Casa Jurnalistului", SUBS),
              ("CNP*WIRED MAGAZINE,", "Wired", SUBS)
             ]
new_expenses = []
for expense in expenses:
    date, _, payee, outflow = expense
    category = ''
    memo = []
    outflow = outflow.replace(",","")
    if "Currency rate" in payee:
        for place in ["NEW YORK", "BROOKLYN", "QUEENS", "MOUNTAIN VIEW", "CLUJ NAPOCA", "JAMAICA", "ORADEA", "LONDON"]:
            if place in payee:
                memo.append("Abroad")
                memo.append(place)
    for substr, name, cat in processors:
        if substr in payee:
            payee = name
            category = cat
            break
    else:
        print(payee)
    if float(outflow) > 100:
        memo.append("expensive")

    new_expenses.append([date, payee, category, ",".join(memo), outflow, 0])
    

for expense in new_expenses:
    print(expense)
with open('cleaned.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Date", "Payee", "Category", "Memo", "Outflow", "Inflow"])
    writer.writerows(new_expenses)


In [ ]:
f = open("./bcge8.csv")

expense_reader = csv.reader(f)
expenses = list(expense_reader)

processors = [("Muller-Schmidig", "Landlord", "Monthly Bills: Rent"), 
              ("Google","Google", "Income for this month"),
              ("Galaxus", "Digitec", "Fun Stuff: Electronics"),
              ("UPC", "UPC", "Monthly Bills: Internet"),
              ("Elektrizitatswerkedes, Kantons Zurich", "EKZ", "Monthly Bills: Electricity"),
              ("Mobility Genossenschaft, GÃ¼tschstrasse 2, 6 ..", "Mobility", "Everyday Expenses: Transportation"),
              ("TERRE DES HOMMES SCHWEIZ", "Terre Des Hommes", "Giving: Charitable"),
              ("SWICA", "Swica", "Everyday Expenses: Medical"),
              ("ZKB", "Transfer: Cash", ""),
              ("VISECA", "Transfer: Credit Card", ""),
              ("Revolut LTD", "Transfer: Revolut", "")
             ]
new_expenses = []
existing = set()
for expense in expenses:
    date, payee, outflow, inflow, memo = expense
    try:
        date = datetime.datetime.strptime(date, "%d.%m.%Y")
    except:
        date = datetime.datetime.strptime(date, "%d.%m.%y")
    date = date.strftime("%m.%d.%Y")
    outflow = outflow.replace("'","")
    inflow = inflow.replace("'","")
    category = ''
    memo = [] if not memo else [memo]
    for substr, name, cat in processors:
        if substr in payee:
            payee = name
            category = cat
            break
    else:
        print(payee)
    value = inflow if inflow else outflow
    if payee in ["Payment order", "Standing order"] and (date, abs(float(value))) in existing:
        continue                 
        
    existing.add((date, abs(float(value))))
    new_expenses.append([date, payee, category, ",".join(memo), outflow, inflow])

new_expenses.sort(key=lambda x: x[0])
    

for expense in new_expenses:
    print(expense)
with open('cleaned.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Date", "Payee", "Category", "Memo", "Outflow", "Inflow"])
    writer.writerows(new_expenses)